<a href="https://colab.research.google.com/github/rkhamilton/vqgan-clip-generator/blob/main/vqgan_clip_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VQGAN_CLIP_GENERATOR**
An easily installable VQGAN+CLIP image generator with a focus on restyling (style transfer) existing videos.  
Documentation is provided at [the project home page](https://github.com/rkhamilton/vqgan-clip-generator).

The vqgan_clip_genrator package was developed primarily for ease of use, ease of installation on local computers, and to facilitate development of a different way to restyle videos.

This package is compatible with all of the usual trained models that work with VQGAN (sflickr, coco, etc). Code is provided here only for ImageNet for simplicity.


In [ ]:
#@title MIT License
# 

# Copyright (c) 2021 Ryan Hamilton

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#@markdown What GPU am I using?

#@markdown V100 > P100 > everything else

!nvidia-smi --query-gpu=gpu_name,memory.total,gpu_bus_id,vbios_version --format=csv
gpu_name = !nvidia-smi --query-gpu=gpu_name, --format=csv

# **Filesystem Setup**
Run either the Local Filesystem or Google Drive blocks to set your root location.

In [ ]:
#@title Connect Google Drive
#@markdown This option connects to your google drive, and saves model files and output to that location for re-use.
import os
abs_root_path = "/content"

from google.colab import drive
drive.mount('/content/drive')

def ensureProperRootPath():
    if len(abs_root_path) > 0:
        os.chdir(abs_root_path) # Changes directory to absolute root path
        print("Root path check: ")
        !pwd

ensureProperRootPath()

def make_folder(folder_name):
  if len(folder_name) > 0:
      path_tmp = os.path.join("/content/drive/MyDrive/",folder_name)
      if not os.path.exists(path_tmp):
          os.mkdir(path_tmp)
      return path_tmp


project_folder_name = "VQGAN_CLIP_GENERATOR" #@param {type: "string"}
abs_root_path = make_folder(project_folder_name)

print("Created folder & set root path to: " + abs_root_path)

In [ ]:
#@title Use Local Colab Filesystem
#@markdown If you use the local google filesystem, all data will be lost at the
#@markdown end of the session. VQGAN models will have to be re-downloaded.
import os
abs_root_path = "/content"
def ensureProperRootPath():
    if len(abs_root_path) > 0:
        os.chdir(abs_root_path) # Changes directory to absolute root path
        print("Root path check: ")
        !pwd

ensureProperRootPath()
print("Your root directory is: ")
!pwd

def make_folder(folder_name):
  abs_root_path = "/content"
  if len(folder_name) > 0:
      path_tmp = abs_root_path + "/drive/MyDrive/" + folder_name
      if not os.path.exists(path_tmp):
          os.mkdir(path_tmp)
      return path_tmp


# **Set Paths and Download Dependencies**
Run either the Local Filesystem or Google Drive blocks to set your root location.

In [ ]:
#@title Download dependencies

!pip install git+https://github.com/openai/CLIP.git
!pip install git+https://github.com/rkhamilton/vqgan-clip-generator.git
!pip install opencv-python scipy
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install git+https://github.com/xinntao/Real-ESRGAN
!pip install sk-video
!pip install opencv-python
!pip install moviepy
!git clone git@github.com:hzwer/arXiv2020-RIFE.git

In [ ]:
#@title Define Paths
extracted_video_frames_dir = os.path.join(abs_root_path, "extracted_video_frames")
generated_video_frames_path = os.path.join(abs_root_path, "generated_video_frames")
upscaled_video_frames_path = os.path.join(abs_root_path, "upscaled_video_frames")

output_dir = os.path.join(abs_root_path, "output")
models_dir = os.path.join(abs_root_path, "models")

make_folder(extracted_video_frames_dir)
make_folder(generated_video_frames_path)
make_folder(upscaled_video_frames_path)
make_folder(output_dir)
make_folder(models_dir)

In [ ]:
#@title Import libraries
from vqgan_clip import generate, video_tools, esrgan
from vqgan_clip.engine import VQGAN_CLIP_Config
import os
from IPython import display
import subprocess

# **Generate Images and Video**

# **Generate a single image from a prompt**

In [16]:
#@title Generate the image
config = VQGAN_CLIP_Config()
output_image_width_pixels = 128 #@param {type: "integer"}
output_image_height_pixels = 128 #@param {type: "integer"}
config.output_image_size = [output_image_width_pixels,output_image_height_pixels]
text_prompts = 'A pastoral landscape painting by Rembrandt' #@param {type: "string"}
iterations = 100 #@param {type: "integer"}

output_filename = os.path.join(output_dir,'my_image.png')
generate.image(eng_config = config,
        text_prompts = text_prompts,
        iterations = iterations,
        output_filename = output_filename)

print(output_filename)
display.display(display.Image(output_filename))

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
Restored from /content/drive/MyDrive/VQGAN_CLIP_GENERATOR/models/vqgan_imagenet_f16_16384.ckpt


100%|██████████| 100/100 [03:20<00:00,  2.00s/iteration]


'A pastoral landscape painting by Rembrandt.png'

In [ ]:
#@title Upscale the image
esrgan.inference_realesrgan(input=output_filename,
        output_images_path=output_dir,
        face_enhance=True)

upscaled_filename = os.path.splitext(output_filename)[0] + '_upscaled.png'
print(upscaled_filename)
display.display(display.Image(upscaled_filename))

# **Generate a zoom video**

In [ ]:
#@title Generate a single image to initialize the video.
#@markdown Generate a video with movement. Every frame that is generated has a shift or zoom applied to it.
#@markdown This gives the appearance of motion in the result.
#@markdown This is one of the most interesting application of VQGAN+CLIP here.

config = VQGAN_CLIP_Config()
text_prompts = 'A pastoral landscape painting by Rembrandt | wolves chasing sheep' #@param {type: "string"}
output_image_width_pixels = 128 #@param {type: "integer"}
output_image_height_pixels = 128 #@param {type: "integer"}
config.output_image_size = [output_image_width_pixels,output_image_height_pixels]
num_video_frames = 150 #@param {type: "integer"}
iterations_per_frame = 15 #@param {type: "integer"}
change_prompts_on_frame= [60, 100] #@param {type: "integer"}
video_framerate=30  #@param {type: "integer"}
zoom_scale=1.02  #@param {type: "number"}
shift_x=1 #@param {type: "integer"}
shift_y=1 #@param {type: "integer"}
final_video_filename = 'zoom_video.mp4' #@param {type: "string"}

output_file_path = os.path.join(output_dir, final_video_filename)


config = VQGAN_CLIP_Config()
config.output_image_size = [output_image_width_pixels,output_image_height_pixels]
init_image = os.path.join(output_dir,'init_image.png')
generate.image(eng_config = config,
        text_prompts = text_prompts,
        iterations = 30,
        output_filename = init_image)

In [ ]:
#@title Generate a zoom video starting from that initial frame.
metadata_comment = generate.video_frames(num_video_frames=num_video_frames,
        eng_config = config,
        text_prompts = text_prompts,
        init_image = init_image,
        generated_video_frames_path = generated_video_frames_path,
        iterations_per_frame = iterations_per_frame,
        change_prompts_on_frame= change_prompts_on_frame,
        zoom_scale=zoom_scale, 
        shift_x=shift_x, 
        shift_y=shift_y,
        z_smoother=True,
        z_smoother_buffer_len=5,
        z_smoother_alpha=0.9)

video_frames_to_encode = generated_video_frames_path

In [ ]:
#@title Upscale the video frames
upscaled_video_frames_path='upscaled_video_frames'
esrgan.inference_realesrgan(input=generated_video_frames_path,
        output_images_path=upscaled_video_frames_path,
        face_enhance=False,
        purge_existing_files=True)
video_frames_to_encode = upscaled_video_frames_path

In [ ]:
#@title Encode the video frames to MP4
video_tools.encode_video(output_file=final_video_filename,
        path_to_stills=video_frames_to_encode,
        metadata_title=text_prompts,
        metadata_comment=metadata_comment,
        input_framerate=video_framerate)

In [ ]:
#@title Use optical flow interpolation with RIFE
RIFE_interpolation_factor = 4 #Valid choices are 4 or 16
of_cmnd = f'python arXiv2020-RIFE\\inference_video.py --exp={2 if RIFE_interpolation_factor==4 else 4} --model=arXiv2020-RIFE\\train_log --video={final_video_filename}'
subprocess.Popen(of_cmnd,shell=True).wait()
print(f'RIFE optical flow command used was:\n{of_cmnd}')
metadata_option = f'-metadata title=\"{text_prompts}\" -metadata comment=\"{metadata_comment}\" -metadata description=\"Generated with https://github.com/rkhamilton/vqgan-clip-generator\"'
# RIFE appends a string to the original filename of the form "original_filename_4X_120fps.mp4"
RIFE_output_filename = os.path.splitext(final_video_filename)[0] + f'_{RIFE_interpolation_factor}X_{video_framerate*RIFE_interpolation_factor}fps.mp4'
FFMPEG_output_filename = os.path.splitext(RIFE_output_filename)[0] + '_reencoded.mp4'
ffmpeg_command = f'ffmpeg -y -i {RIFE_output_filename} -vcodec libx264 -crf 23 -pix_fmt yuv420p -hide_banner -loglevel error {metadata_option} {FFMPEG_output_filename}'
subprocess.Popen(ffmpeg_command,shell=True).wait()
print(f'FFMPEG command used was:\t{ffmpeg_command}')

# **Style transfer to an existing video**

In [ ]:
#@title Setup
#@markdown An existing video will have a VQGAN style applied to each frame.
#@markdown Refer to the [documentation](https://github.com/rkhamilton/vqgan-clip-generator/blob/main/README.md) 
#@markdown for an explanation of the options.

config = VQGAN_CLIP_Config()
input_video_path = 'input_video.MOV' #@param {type: "string"}
output_video_filename = 'output_video.mp4' #@param {type: "string"}
video_framerate = 30 #@param {type: "integer"}
text_prompts = 'portrait on deviantart' #@param {type: "string"}
output_image_width_pixels = 128 #@param {type: "integer"}
output_image_height_pixels = 128 #@param {type: "integer"}
iterations_per_frame = 50 #@param {type: "integer"}
change_prompts_on_frame = [70, 150]  #@param {type: "list"}
current_source_frame_image_weight = 2.0 #@param {type: "number"}
current_source_frame_prompt_weight=0.5 #@param {type: "number"}
copy_audio=False #@param {type: "boolean"}

config.output_image_size = [output_image_width_pixels,output_image_height_pixels]
print(input_video_path)

final_output_filename = os.path.join(output_dir,'output.mp4')
copy_audio = True
extraction_framerate = 30
output_framerate = 60

init_image = os.path.join(output_dir,'init_image.png')
output_video_path = os.path.join(output_dir,output_video_filename)

input_video.MOV
Extracting image frames from original video


Style Transfer:   0%|          | 0/151 [00:00<?, ?image/s]

In [ ]:
#@title Extract frames and style transfer to still images
original_video_frames = video_tools.extract_video_frames(input_video_path, 
        extraction_framerate = video_framerate)

In [ ]:
#@title Generate a single image to initialize the video.
generate.image(eng_config = config,
        text_prompts = text_prompts,
        init_image=original_video_frames[0],
        iterations = 10,
        output_filename = init_image)

In [ ]:
#@title Apply a style to the extracted video frames.
metadata_comment = generate.style_transfer(original_video_frames,
        eng_config=config,
        current_source_frame_image_weight=current_source_frame_image_weight, # effects how closely the output video will be to the source video. Values of 0.5-8.0 are reasonable.
        text_prompts = text_prompts,
        iterations_per_frame = iterations_per_frame,
        change_prompts_on_frame=change_prompts_on_frame,
        init_image=init_image,
        generated_video_frames_path = generated_video_frames_path,
        current_source_frame_prompt_weight=current_source_frame_prompt_weight,
        z_smoother=True,
        z_smoother_alpha=0.7,
        z_smoother_buffer_len=3)

video_frames_to_encode = generated_video_frames_path

In [ ]:
#@title Upscale the video frames
face_enhance = False #@param {type: "boolean"}
esrgan.inference_realesrgan(input=generated_video_frames_path,
        output_images_path=upscaled_video_frames_path,
        face_enhance=face_enhance,
        purge_existing_files=True)
video_frames_to_encode = upscaled_video_frames_path

In [ ]:
#@title Use a wrapper for FFMPEG to encode the video.
generated_video_no_audio=os.path.join(output_dir,'output_no_audio.mp4')
video_tools.encode_video(output_file=generated_video_no_audio,
        path_to_stills=video_frames_to_encode,
        metadata_title=text_prompts,
        metadata_comment=metadata_comment,
        input_framerate=video_framerate)

In [ ]:
#@title Use optical flow interpolation with RIFE
RIFE_interpolation_factor = 4 #Valid choices are 4 or 16
of_cmnd = f'python arXiv2020-RIFE\\inference_video.py --exp={2 if RIFE_interpolation_factor==4 else 4} --model=arXiv2020-RIFE\\train_log --video={final_video_filename}'
subprocess.Popen(of_cmnd,shell=True).wait()
print(f'RIFE optical flow command used was:\n{of_cmnd}')
metadata_option = f'-metadata title=\"{text_prompts}\" -metadata comment=\"{metadata_comment}\" -metadata description=\"Generated with https://github.com/rkhamilton/vqgan-clip-generator\"'
# RIFE appends a string to the original filename of the form "original_filename_4X_120fps.mp4"
RIFE_output_filename = os.path.splitext(final_video_filename)[0] + f'_{RIFE_interpolation_factor}X_{video_framerate*RIFE_interpolation_factor}fps.mp4\"'
FFMPEG_output_filename = os.path.splitext(RIFE_output_filename)[0] + '_reencoded.mp4\"'
ffmpeg_command = f'ffmpeg -y -i {RIFE_output_filename} -vcodec libx264 -crf 23 -pix_fmt yuv420p -hide_banner -loglevel error {metadata_option} {FFMPEG_output_filename}'
subprocess.Popen(ffmpeg_command,shell=True).wait()
print(f'FFMPEG command used was:\t{ffmpeg_command}')
generated_video_no_audio = FFMPEG_output_filename

In [ ]:
#@title Copy audio from the original file
if copy_audio:
        video_tools.copy_video_audio(input_video_path, generated_video_no_audio, final_output_filename)
        os.remove(generated_video_no_audio)
else:
        os.rename(generated_video_no_audio,final_output_filename)

In [ ]:
#@title Download video
from google.colab import files
files.download(output_video_path)